In [42]:
import numpy as np
import pandas as pd

In [43]:
link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2018'


In [49]:

df1 = pd.read_html(link,header=0)[2]
df2 = pd.read_html(link,header=0)[3]
df3 = pd.read_html(link,header=0)[4]
df4 = pd.read_html(link,header=0)[5]

In [50]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [51]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]
...,...,...,...,...,...,...
264,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,[163]
265,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[137]
266,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[226]
267,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",[258]


In [52]:
from tmdbv3api import TMDb
import json
import requests


tmdb = TMDb()
tmdb.api_key = '7767da907c5a44d336bd201316cdb8ab'

In [53]:
from tmdbv3api import Movie
tmdb_movie = Movie()

In [54]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [55]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [56]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],Drama History Western
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],Action Thriller
...,...,...,...,...,...,...,...
264,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,[163],Mystery Adventure Comedy Crime
265,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[137],Thriller Science Fiction Action Adventure
266,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[226],Drama History
267,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",[258],Thriller Crime Drama Action


In [57]:
df_2018 = df[['Title','Cast and crew','genres']]
df_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller
...,...,...,...
264,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
265,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure
266,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History
267,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action


In [58]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [59]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [60]:
df_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...,...
264,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
265,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
266,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
267,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [61]:
df_2018 = df_2018.rename(columns={'Title':'movie_title'})
new_df18 = df_2018.loc[:,['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name',]]

In [62]:
new_df18.isnull().sum()

movie_title       0
genres            3
director_name     0
actor_1_name      0
actor_2_name      9
actor_3_name     26
dtype: int64

In [63]:
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')
new_df18['movie_title'] = new_df18['movie_title'].str.lower()

In [64]:
new_df18.isnull().sum()

movie_title      0
genres           3
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
dtype: int64

In [65]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']

In [66]:
new_df18

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Bryan Brown Sam Neill unknown Warwick Thornton...
4,the commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
264,holmes & watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Will Ferrell John C. Reilly Rebecca Hall Etan ...
265,vice,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Christian Bale Amy Adams Steve Carell Adam McK...
266,on the basis of sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Felicity Jones Armie Hammer Justin Theroux Mim...
267,destroyer,Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Nicole Kidman Sebastian Stan Toby Kebbell Kary...


In [67]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [68]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [69]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
237,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
238,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[221]
239,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures / Ent...,Sam Mendes (director/screenplay); Krysty Wilso...,[222]
240,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[223]


In [70]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))
df_2019 = df[['Title','Cast and crew','genres']]
df_2019

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Thriller Action Mystery Adventure Horror
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama
...,...,...,...
237,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family
238,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance
239,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action Thriller
240,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History


In [71]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [72]:
df_2019

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Thriller Action Mystery Adventure Horror,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman
...,...,...,...,...,...,...,...
237,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones
238,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh
239,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong
240,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson


In [73]:
df_2019 = df_2019.rename(columns={'Title':'movie_title'})
new_df19 = df_2019.loc[:,['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name',]]

In [74]:
new_df19.isnull().sum()

movie_title       0
genres            0
director_name     0
actor_1_name      0
actor_2_name      1
actor_3_name     15
dtype: int64

In [75]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

In [76]:
new_df19['movie_title'] = new_df19['movie_title'].str.lower()

In [77]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']

In [78]:
new_df19

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,escape room,Thriller Action Mystery Adventure Horror,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Taylor Russell Logan Miller Deborah Ann Woll A...
1,rust creek,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,american hangman,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Donald Sutherland Vincent Kartheiser Oliver De...
3,a dog's way home,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Bryce Dallas Howard Edward James Olmos Alexand...
4,the upside,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Bryan Cranston Kevin Hart Nicole Kidman Neil B...
...,...,...,...,...,...,...,...
237,spies in disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Will Smith Tom Holland Rashida Jones Nick Brun...
238,little women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Saoirse Ronan Emma Watson Florence Pugh Greta ...
239,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,George MacKay Dean-Charles Chapman Mark Strong...
240,just mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [79]:
my_df = new_df18.append(new_df19,ignore_index=True)
my_df

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Bryan Brown Sam Neill unknown Warwick Thornton...
4,the commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
506,spies in disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Will Smith Tom Holland Rashida Jones Nick Brun...
507,little women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Saoirse Ronan Emma Watson Florence Pugh Greta ...
508,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,George MacKay Dean-Charles Chapman Mark Strong...
509,just mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [80]:
old_df = pd.read_csv('new_data.csv')

In [81]:
old_df

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5359,the incredible jessica james,Romance Comedy,Jim Strouse,Jessica Williams,Chris O'Dowd,Keith Stanfield,Jessica Williams Chris O'Dowd Keith Stanfield ...
5360,can't buy my love,Romance,Farhad Mann,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Adelaide Kane Benjamin Hollingsworth Jean Loui...
5361,thick lashes of lauri mäntyvaara,Romance Comedy,Hannaleena Hauru,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Inka Haapamäki Rosa Honkonen Tiitus Rantala Ha...
5362,cop and a half: new recruit,Crime Comedy Action Family,Jonathan A. Rosenbaum,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [82]:
final_df = old_df.append(my_df,ignore_index=True)

In [83]:
final_df

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,CCH Pounder Joel David Moore Wes Studi James C...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5870,spies in disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Will Smith Tom Holland Rashida Jones Nick Brun...
5871,little women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5872,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,George MacKay Dean-Charles Chapman Mark Strong...
5873,just mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [84]:
final_df.isnull().sum()

movie_title      0
genres           3
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
comb             3
dtype: int64

In [85]:
final_df = final_df.dropna(how='any')

final_df.isna().sum()

movie_title      0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
comb             0
dtype: int64

In [86]:
final_df.to_csv('final_data.csv',index=False)